In [ ]:
import requests

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow



In [ ]:

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# data engineering

In [ ]:

my_wm_username = 'wli10'
search_url = 'http://buckets.peterbeshai.com/api/?player=201939&season=2014'
response = requests.get(search_url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"})

url = 'http://buckets.peterbeshai.com/api/?player=201939&season=2015'
data = requests.get(url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"})


    

cor =[]    
made = []

for shot in response.json():
    cor.append((shot['LOC_X'],shot['LOC_Y']))
    made.append(shot['EVENT_TYPE'])
    

for shots in data.json():
    cor.append((shots['LOC_X'],shots['LOC_Y']))
    made.append(shots['EVENT_TYPE'])
      
len(cor),len(made)


In [ ]:
for i in range(len(made)):
    if made[i] == 'Made Shot':
        made[i] =1
    else:
        made[i] =0
len(made)


In [ ]:
for val in cor:
    print(val)

In [ ]:
x = [np.array(list(val)) for val in cor]
x = np.array(x).T
x.shape

In [ ]:
y = np.array(made,dtype=np.uint8).reshape(-1,len(made))
y.shape

In [ ]:
n = x.shape[1]*3//4
X_train = x[:,0:n]/712
X_test = x[:,n:]/712
y_train = y[:,0:n]
y_test = y[:,n:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train

In [ ]:
X_train, cor[0:10]

In [ ]:
X_train.max(),X_train.min(), y_train


In [ ]:
X_train[0, 0:211].shape,X_train[1, 0:211].shape,y_train[:,0:211].shape

In [ ]:
y_new = y_train[:,0:211]
y_new

In [ ]:
plt.scatter(X_train[0, 0:211], X_train[1, 0:211],  c = y_new, s=40, cmap=plt.cm.Spectral)



# function define

In [ ]:
def sigmoid(X):
    s = 1 / (1+np.exp(-X))
    return s


def relu(X):
    s = np.maximum(0,X)
    return s

def tanh(X):
    s = (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))
    return s
    
def initialize_parameters(layer_dims):
    
    parameters = {}
    L = len(layer_dims)
    for i in range(1,L):
        parameters['W'+str(i)] = np.random.randn(layer_dims[i],layer_dims[i-1])
        parameters['b'+str(i)] = np.zeros((layer_dims[i],1))
        
        assert(parameters['W' + str(i)].shape == (layer_dims[i], layer_dims[i-1]))
        assert(parameters['b' + str(i)].shape == (layer_dims[i], 1))        
    return parameters
        
        
def forward_prop(X,parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    
    W4 = parameters['W4']
    b4 = parameters['b4']
    
    
    
    Z1 = np.dot(W1,X)+b1
    A1 = tanh(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = tanh(Z2)
    Z3 = np.dot(W3,A2)+b3
    A3 = tanh(Z3)
    
    Z4 = np.dot(W4,A3)+b4
    A4 = sigmoid(Z4)
    
    cache = (Z1,A1,W1,b1,Z2,A2,W2,b2,Z3,A3,W3,b3,Z4,A4,W4,b4)
    return A4,cache


def backward_prop(X,y,cache):
    m = X.shape[1]
    (Z1,A1,W1,b1,Z2,A2,W2,b2,Z3,A3,W3,b3,Z4,A4,W4,b4) = cache
    
    dZ4 = A4-y
    dW4 = 1./m * np.dot(dZ4,A3.T)
    db4 = 1./m * np.sum(dZ4, axis =1, keepdims = True)
    
    dA3 = np.dot(W4.T,dZ4)
    dZ3 = np.multiply(dA3 ,(1-A3**2))
    dW3 = 1. /m * np.dot(dZ3,A2.T)
    db3 = 1. /m * np.sum(dZ3,axis=1,keepdims=True)
    
    dA2 = np.dot(W3.T,dZ3)
    dZ2 = dA2 * (1-A2**2)
    dW2 = 1. /m * np.dot(dZ2,A1.T)
    db2 = 1. /m * np.sum(dZ2,axis=1,keepdims=True)
    

    dA1 = np.dot(W2.T,dZ2)
    dZ1 = dA1 * (1-A1**2)
    dW1 = 1. /m * np.dot(dZ1,X.T)
    db1 = 1. /m * np.sum(dZ1,axis=1,keepdims=True)
    
    gradients = {"dZ4": dZ4, "dW4": dW4, "db4": db4,
                 "dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients


def update_para(gradients,parameters,learning_rate):
    n = len(parameters) //2
    for i in range(n):
        parameters['W'+str(i+1)] = parameters['W'+str(i+1)] - learning_rate*gradients['dW'+str(i+1)]
        parameters['b'+str(i+1)] = parameters['b'+str(i+1)] - learning_rate*gradients['db'+str(i+1)]
        
    return parameters

def predict(X,y,parameters):
    m = X.shape[1]
    
    A4,cache = forward_prop(X,parameters)
    
    p = np.zeros((1,m))
    for i in range(0,A4.shape[1]):
        if A4[0,i] > 0.5:
            p[0,i] =1
        else:
            p[0,i] =0
    print('accuracy rate is: ',np.mean((p[0,:]==y[0,:])))
    
    return p

def compute_cost(A4,y):
    m = y.shape[1]
    loss = -(y*np.log(A4)+(1-y)*np.log(1-A4))
    cost = 1./m * np.nansum(loss)
    
    return cost

def get_batch(X,y,batch_size = 16):
    m = X.shape[1]
    perm = np.random.permutation(m)
    X_perm = X[:,perm]
    y_perm = y[:,perm]
    
    n = np.int(m/batch_size)
    batches = []
    for i in range(n):
        X_batch = X_perm[:,i*batch_size:(i+1)*batch_size]
        y_batch = y_perm[:,i*batch_size:(i+1)*batch_size]
        batches.append((X_batch,y_batch))
    if m%batch_size != 0:
        X_batch = X_perm[:,n*batch_size:]
        y_batch = y_perm[:,n*batch_size:]       
        batches.append((X_batch,y_batch))
    return batches
    
def predict_dec(X,parameters):
    a4, cache = forward_prop(X,parameters)
    predictions = (a4>0.5)
    
    return predictions

def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)
    plt.show()

In [ ]:
np.int(32/5)

In [ ]:
np.random.permutation(10)

In [ ]:
layer_dims =[X.shape[0],32,20,3,1]
parameters = initialize_parameters(layer_dims)
len(parameters)

In [ ]:
def model(X, Y, learning_rate = 0.3, num_iterations = 30001, print_cost = True, lambd = 0, keep_prob = 1):
    grads = {}
    costs = []
    m = X.shape[1]
    layer_dims =[X.shape[0],32,20,3,1]
    
    parameters = initialize_parameters(layer_dims)
    
    
    for i in range(0,num_iterations):
        
        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        if keep_prob ==1:
            A4,cache = forward_prop(X, parameters)
        elif keep_prob < 1:
            A4,cache = forward_prop_with_dropout(X, parameters)
            
        # cost function
     
        if lambd == 0:
            cost = compute_cost(A4, Y)
        else:
            cost = compute_cost_with_regularization(A4, Y, parameters, lambd)
            
               # Backward propagation.
        assert(lambd==0 or keep_prob==1) 
        
        
        if lambd == 0 and keep_prob == 1:
            grads = backward_prop(X, Y, cache)
        elif lambd != 0:
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)        
   
                
       # grads = backward_propagation(X, Y, cache)
        parameters = update_para(grads,parameters,learning_rate)
    
        if print_cost and i%10000 == 0:
            print('Cost after iteration {}: {}'.format(i,cost))
        if print_cost and i%1000 == 0:
            costs.append(cost)
            
    plt.plot(costs)
    plt.xlabel('cost')
    plt.ylabel('iteration (X1000)')
    plt.title('Learning rate = ' +str(learning_rate))
    plt.show()

    return parameters

In [ ]:
def model_pure(X, Y, learning_rate = 0.3, num_iterations = 3000, print_cost = True):
    grads = {}
    costs = []
    m = X.shape[1]
    layer_dims =[X.shape[0],32,20,3,1]
    
    parameters = initialize_parameters(layer_dims)
    
    
    for i in range(0,num_iterations):
        
        batches = get_batch(X,Y,batch_size = 32)
        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        for b in range(len(batches)):
            X_batch, y_batch = batches[b]
            A4,cache = forward_prop(X_batch, parameters)
            cost = compute_cost(A4, y_batch)
            grads = backward_prop(X_batch, y_batch, cache)
            parameters = update_para(grads,parameters,learning_rate)
    
        if print_cost and i%1000 == 0:
            print('Cost after iteration {}: {}'.format(i,cost))
        if print_cost and i%100 == 0:
            costs.append(cost)
            
    plt.plot(costs)
    plt.xlabel('cost')
    plt.ylabel('iteration (X1000)')
    plt.title('Learning rate = ' +str(learning_rate))
    plt.show()

    return parameters

In [ ]:
a = np.array([1, 2, 3])
a.shape

In [ ]:
parameters = model_pure(X_train, y_train,learning_rate = 0.003)
print ("On the training set:")
predictions_train = predict(X_train, y_train, parameters)
print ("On the test set:")
predictions_test = predict(X_test, y_test, parameters)

# KERAS

In [ ]:
def model(input_shape):
    X_input = Input(input_shape)
    X = Dense(128,activation='tanh')(X_input)
    X = BatchNormalization(axis=-1)(X)
    X = Dense(64,activation='tanh')(X)
    X = BatchNormalization(axis=-1)(X)
    X = Dense(32,activation='tanh')(X)
    X = BatchNormalization(axis=-1)(X)
    X = Dense(32,activation='tanh')(X)
    X = BatchNormalization(axis=-1)(X)
    X = Dense(16,activation='tanh')(X)
    X = BatchNormalization(axis=-1)(X)
    X = Dense(8,activation='tanh')(X)
    X = BatchNormalization(axis=-1)(X)
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=X_input,outputs=X)
    
    return model
    

In [ ]:
S_model = model((2,))

In [ ]:
S_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
X = X_train.T
Y = y_train.T

In [ ]:
S_model.fit(X,Y,epochs=30,batch_size=16)